# Melyik a nyerő csapat, kik a nyerő kutyák?

- FlyBall versenyen nagyon fontos a részvétel és a játék, de a csapatok a győzelemért futnak. 
- Vizsgáljuk meg, hogy melyik versenyen kik nyertek és hogy a WR kutyák között kik szerepelnek a győztes csapatokban.
- Milyen tulajdonságuk játszik nagyobb szerepet a győztes kutyáknak, csapatoknak?

In [1]:
import pandas as pd

#Betöltjük az összes verseny adatát tartalmazó tisztitott táblázatot
winner = pd.read_csv('./../data/szines2betti.csv', index_col=[0])

In [23]:
winner.shape

(33627, 17)

Divíziók vizsgálata:
- a versenyeken nagy részén ugyanazok a diviziók vesznek részt első és második nap
- kivétel a júl.29-30-i bajnokság, ott mindkét nap versenynap, így a diviziók a két napon szétosztva szerepelnek
- a júl.15-16-i és a szept 2-3-i részben átfed
- Domain kérdés-válasz: ahol ugyanazok mennek mindkét nap, ott az az első nap időmérő, a második nap éles verseny
- ez az adatokból is látszik: az időmérő napokon a váltók ugyanannyiszor futnak, míg a verseny napokon a győztes csapatok futják általában a legtöbbet

In [78]:
#Dátumonként a diviziók
winner[['place', 'date', 'Division']].groupby(['date', 'Division' ]).count()

place
date       Division       
2023-04-22 1.0         244
           2.0         189
2023-04-23 1.0         279
           2.0         218
2023-05-06 1.0         390
...                    ...
2023-10-15 4.0         390
           5.0         454
           6.0         448
           7.0         273
           8.0         303

[109 rows x 1 columns]

In [4]:
#Minden divizióban többféle akadálymagasságon futnák:
winner[['Division', 'Hurdles']].groupby(['Division', 'Hurdles']).count().tail()

Empty DataFrame
Columns: []
Index: [(13.0, 15.0), (13.0, 17.5), (13.0, 20.0), (13.0, 25.0), (13.0, 27.5)]

In [5]:
#A csapatok szereplései a verseny napokon (Race) az egyes diviziókon belül:

winner[winner['Q/Race'] == 'Race'][['who', 'name', 'Division','place']].groupby(
    ['place', 'Division', 'who']).count().head()


name
place           Division who                         
BCCCZ_Prostejov 1.0      Flyvaryors New Wave       64
                         Flyvaryors Traditional    52
                         WildRunners Maximini      48
                         WildRunners dreamers      24
                         WildRunners hopefuls      60

In [6]:
#A csapatok szereplései a verseny napokon (Race) az egyes diviziókon belül győztes/vesztes futamokra bontva:
winner[winner['Q/Race'] == 'Race'][['who', 'Division','place', 'W/L/T', 'name', 'rank']].groupby(
    ['place', 'Division', 'who', 'W/L/T']).count().head(10)

name  rank
place           Division who                    W/L/T            
BCCCZ_Prostejov 1.0      Flyvaryors New Wave    L        28    28
                                                W        36    36
                         Flyvaryors Traditional L        16    16
                                                W        36    36
                         WildRunners Maximini   L        24    24
                                                W        24    24
                         WildRunners dreamers   L        24    24
                         WildRunners hopefuls   L        32    32
                                                W        28    28
                2.0      FOFR DOG               L        32    32

In [2]:
#Ötlet: W/L/T számolással jobban kiderül ki nyerhette az adott hely versenynapján az egyes diviziókat:

#táblázatba mentjük a groupby eredményét:
who_won = winner[winner['Q/Race'] == 'Race'][['who', 'Division','place', 'W/L/T', 'name']].groupby(
    ['place', 'Division', 'who', 'W/L/T']).count()


In [3]:
#A groupby tábla multiindexét átalakítva:

who_won.reset_index(drop=False, inplace=True)

In [4]:
who_won.head(20)

,place,Division,who,W/L/T,name
0,BCCCZ_Prostejov,1.0,Flyvaryors New Wave,L,28
1,BCCCZ_Prostejov,1.0,Flyvaryors New Wave,W,36
2,BCCCZ_Prostejov,1.0,Flyvaryors Traditional,L,16
3,BCCCZ_Prostejov,1.0,Flyvaryors Traditional,W,36
4,BCCCZ_Prostejov,1.0,WildRunners Maximini,L,24
5,BCCCZ_Prostejov,1.0,WildRunners Maximini,W,24
6,BCCCZ_Prostejov,1.0,WildRunners dreamers,L,24
7,BCCCZ_Prostejov,1.0,WildRunners hopefuls,L,32
8,BCCCZ_Prostejov,1.0,WildRunners hopefuls,W,28
9,BCCCZ_Prostejov,2.0,FOFR DOG,L,32


In [10]:
#A győztes (W) sorok kiválogatása:
who_wonW = who_won[who_won['W/L/T'] == 'W']
who_wonW.shape

(357, 5)

In [11]:
#Helyszínek listába az elemző kódhoz:
helyszinek = who_wonW['place'].unique()

In [12]:
#Helyezéseket kiosztó elemző kód:

A = []
for i in helyszinek:
    
    L = len(who_wonW[who_wonW['place'] == i]['Division'].value_counts()) #melyik helyszínen hány divizió volt
    
    for j in range(L+1):
        
        sor = who_wonW[(who_wonW['Division'] == j)&(who_wonW['place'] == i)]['name']
        
        x = sor.rank(method='min', ascending=False)
        
        A.append(x)


In [13]:
#Az eredmények dataframe-ben jelennek meg, a 'name' oszlop alatt az adott helyszín diviziójában elért helyezés látható,
#az index megmaradt a who_won táblázatból

series = pd.concat(A, axis=0)
df = pd.DataFrame(series)

In [14]:
df

,name
1,1.0
3,1.0
5,4.0
8,3.0
11,2.0
...,...
740,4.0
742,5.0
744,2.0
746,1.0


In [16]:
#Egyesítjük az eredményeket a who_wonW táblával:
ranks_df = pd.merge(df, who_wonW, left_index=True, right_index=True)
ranks_df #357 sor lett, mint a df

,name_x,place,Division,who,W/L/T,name_y
1,1.0,BCCCZ_Prostejov,1.0,Flyvaryors New Wave,W,36
3,1.0,BCCCZ_Prostejov,1.0,Flyvaryors Traditional,W,36
5,4.0,BCCCZ_Prostejov,1.0,WildRunners Maximini,W,24
8,3.0,BCCCZ_Prostejov,1.0,WildRunners hopefuls,W,28
11,2.0,BCCCZ_Prostejov,2.0,FOFR DOG,W,28
...,...,...,...,...,...,...
740,4.0,Tápiószentmárton,13.0,BayernXpress I,W,20
742,5.0,Tápiószentmárton,13.0,Blizzard from OS,W,16
744,2.0,Tápiószentmárton,13.0,JokerZ Lou,W,36
746,1.0,Tápiószentmárton,13.0,Real Ballance Flyball Team,W,48


In [17]:
#ranks_df-ből a WR csapathelyezésekhez a WildRunners csapatokat kiszűrjük
#a a csapatokban résztvevő kutyákhoz szeretnénk kötni, ezért join: ranks_df + winner_race, "on": oszlop lista

In [18]:
#kiválogatjuk a WildRunners csapatokat:
ranks_df_WR = ranks_df[ranks_df['who'].str.startswith('WildRunners')].copy()

#sorba rendezzük a diviziók szerint
ranks_df_WR.sort_values('place')

,name_x,place,Division,who,W/L/T,name_y
5,4.0,BCCCZ_Prostejov,1.0,WildRunners Maximini,W,24
8,3.0,BCCCZ_Prostejov,1.0,WildRunners hopefuls,W,28
17,1.0,BCCCZ_Prostejov,2.0,WildRunners Nightfall,W,56
19,2.0,BCCCZ_Prostejov,2.0,WildRunners Zephyr,W,28
29,4.0,BCCCZ_Prostejov,3.0,WildRunners Wonders,W,16
31,1.0,BCCCZ_Prostejov,3.0,WildRunners XXL,W,40
59,3.0,Budeckej_Vodnany,2.0,WildRunners dreamers,W,36
90,2.0,Budeckej_Vodnany,4.0,WildRunners Maximini,W,36
145,5.0,Budeckej_Vodnany,8.0,WildRunners Wonders,W,8
208,6.0,CrazyCup_NoveMesto,6.0,WildRunners XXL,W,8


In [19]:
#Melyik versenyen milyen diviziókban milyen helyezéseket (name_x) gyűjtöttek a WildRunners csapatok?
ranks_df_WR[['name_x', 'place', 'Division', 'who']].groupby(['place', 'Division', 'name_x' ]).count()

who
place              Division name_x     
BCCCZ_Prostejov    1.0      3.0       1
                            4.0       1
                   2.0      1.0       1
                            2.0       1
                   3.0      1.0       1
                            4.0       1
Budeckej_Vodnany   2.0      3.0       1
                   4.0      2.0       1
                   8.0      5.0       1
CrazyCup_NoveMesto 1.0      4.0       1
                   2.0      1.0       1
                   3.0      2.0       1
                   4.0      1.0       1
                   5.0      4.0       1
                   6.0      6.0       1
EagersCup_Zamberk  1.0      7.0       1
                   2.0      4.0       1
                   4.0      2.0       1
                   5.0      6.0       1
                   8.0      5.0       1
                            6.0       1
Flyvarors          1.0      3.0       1
                   2.0      3.0       1
                   4.0      4.0       1
Heroltice          1.0      1.0       1
                            2.0       2
                            4.0       1
                   2.0      1.0       1
                            4.0       1
Krypky_Litomsyl    1.0      5.0       1
                   2.0      4.0       1
                   3.0      4.0       1
                   4.0      5.0       1
Machale_Luonovice  1.0      5.0       1
                   2.0      1.0       1
                   4.0      3.0       1
Mistrovsy_Blsany   1.0      7.0       1
                   2.0      4.0       1
                            6.0       1
                            8.0       1
Tápiószentmárton   3.0      5.0       1
                   8.0      7.0       1
                   9.0      1.0       1
                   13.0     3.0       1

In [20]:
#A name_x alatt a helyezések, mellette ezek számossága látható
ranks_df_WR[['name_x', 'Division']].groupby('name_x').count()

,Division
name_x,
1.0,8
2.0,6
3.0,6
4.0,11
5.0,6
6.0,4
7.0,3
8.0,1


## Szeretnénk látni, hogy mely WR kutyák értek el helyezést.
A WR csapatok tagjai egy adott versenyen állandó csapatot alkotnak, de versenyről versenyre változhatnak a csapatok tagjaik,  ezért ahhoz, hogy kutyánként is lássuk a helyezéseket, összekötjük ezt a kettő táblát:
- ranks_df
- winner_race

In [38]:
#ezt a táblát
ranks_df.shape #357 sor

(357, 6)

In [21]:
ranks_df.head(2)

,name_x,place,Division,who,W/L/T,name_y
1,1.0,BCCCZ_Prostejov,1.0,Flyvaryors New Wave,W,36
3,1.0,BCCCZ_Prostejov,1.0,Flyvaryors Traditional,W,36


In [24]:
#ezzel a táblával 
winner.head(2) #33ezer sor

,Division,when,colour,competition,date,place,who,with who,total time,W/L/T,Hurdles,rank,name,relay,time,long_date,Q/Race
0,13.0,12:26:30,Blue,FLYBALL_EFC,2023-07-29,Tápiószentmárton,Real Ballance Flyball Team,WildRunners Zephyr,21.18,L,15.0,1,Chase,0.33,4.21,2023-07-29 12:26:30,Race
1,13.0,12:26:30,Red,FLYBALL_EFC,2023-07-29,Tápiószentmárton,WildRunners Zephyr,Real Ballance Flyball Team,20.49,W,25.0,1,Brix,error,4.58,2023-07-29 12:26:30,Race


In [25]:
winner[winner['Q/Race'] == 'Race'].shape #19912

(19912, 17)

In [32]:
#és még nézzük meg, hogy a 19912-ből mennyi a W:
winner_race = winner[ ( winner['Q/Race'] == 'Race') & (winner['W/L/T'] == 'W') ] #(ha 9953, akkor jó lesz, OK!!)

In [33]:
# on: ide több oszlop listája jön, azok és olyan sorrendben, ahogy a groupby történt

kutya_rank = pd.merge(ranks_df, winner_race, on=['place', 'Division', 'who', 'W/L/T'])

In [35]:
kutya_rank.shape #9953 sor

(9953, 19)

In [36]:
wr_kutya_rank = kutya_rank[kutya_rank['who'].str.startswith('WildRunners')].copy()
wr_kutya_rank['place'].unique() # 10 verseny

array(['BCCCZ_Prostejov', 'Budeckej_Vodnany', 'CrazyCup_NoveMesto',
       'EagersCup_Zamberk', 'Flyvarors', 'Heroltice', 'Krypky_Litomsyl',
       'Machale_Luonovice', 'Mistrovsy_Blsany', 'Tápiószentmárton'],
      dtype=object)

In [37]:
wr_kutya_rank.shape #1109 sor

(1109, 19)

In [60]:
wr_kutya_rank.head(1)

,name_x,place,Division,who,W/L/T,name_y,when,colour,competition,date,with who,total time,Hurdles,rank,name,relay,time,long_date,Q/Race
72,4.0,BCCCZ_Prostejov,1.0,WildRunners Maximini,W,24,09:58:49,Blue,FLYBALL_CZ,2023-06-25,WildRunners hopefuls,18.4,25.0,1,Bree,0.15,5.12,2023-06-25 09:58:49,Race


In [39]:
kutyak_helyezese = wr_kutya_rank[['name','name_x', 'place', 'Division']].groupby(['name_x', 'place', 'Division','name',]).count()

In [40]:
kutyak_helyezese.reset_index(drop=False, inplace=True)

In [41]:
kutyak_helyezese.head()

,name_x,place,Division,name
0,1.0,BCCCZ_Prostejov,2.0,Alex
1,1.0,BCCCZ_Prostejov,2.0,Fleur
2,1.0,BCCCZ_Prostejov,2.0,Gerry
3,1.0,BCCCZ_Prostejov,2.0,Nina
4,1.0,BCCCZ_Prostejov,2.0,Sara


In [42]:
kutyak_helyezese.groupby(['name_x']).count().head(50)

,place,Division,name
name_x,,,
1.0,37,37,37
2.0,28,28,28
3.0,28,28,28
4.0,48,48,48
5.0,26,26,26
6.0,20,20,20
7.0,13,13,13
8.0,4,4,4


In [ ]:
# 37 kutyánál szerepel első hely, 28 kutyánál második hely.
#Itt lehetne az összehasonlítás alapja, hogy a kutyáknak a helyezésekhez pontot adunk:
#8.hely 1 pont. 1.hely 8 pont.

In [43]:
#Dictionary a pontok kiosztásához
dict_points = {1.0 : 8, 
              2.0 : 7,
              3.0 : 6,
              4.0 : 5,
              5.0 : 4,
              6.0 : 3,
              7.0 : 2,
              8.0 : 1
              }

In [44]:
#A kutyákhoz rendeljük a pontokat:
mapping = dict_points
kutyak_helyezese['rank_points'] = kutyak_helyezese.name_x.map(mapping)

In [59]:
kutyak_helyezese.head(10)

,name_x,place,Division,name,rank_points
0,1.0,BCCCZ_Prostejov,2.0,Alex,8
1,1.0,BCCCZ_Prostejov,2.0,Fleur,8
2,1.0,BCCCZ_Prostejov,2.0,Gerry,8
3,1.0,BCCCZ_Prostejov,2.0,Nina,8
4,1.0,BCCCZ_Prostejov,2.0,Sara,8
5,1.0,BCCCZ_Prostejov,3.0,Bruno,8
6,1.0,BCCCZ_Prostejov,3.0,Negro,8
7,1.0,BCCCZ_Prostejov,3.0,Rocky,8
8,1.0,BCCCZ_Prostejov,3.0,Scotty,8
9,1.0,BCCCZ_Prostejov,3.0,Speedy,8


In [53]:
#Kutyákat neveik szerint csoportosítunk
kutymok = kutyak_helyezese[['name','rank_points']].groupby('name').sum().sort_values('rank_points', ascending=False)
kutymok.head(1)

,rank_points
name,
Bree,50


In [87]:
### Helyszínek szerint csoportosítunk és elmentjük külön táblázatba
kutya_helyszin = kutyak_helyezese[['place','rank_points']].groupby('place').sum().sort_values('rank_points', ascending=False).reset_index()

#Új oszlop : hányan voltak azon a versenyen:
kutya_team = [39, 30, 29, 30, 24, 16, 16, 20, 17, 26]
kutya_helyszin['kutya_team'] = kutya_team

#Pontok vetítése egy kutyára új oszlopba, a jobb olvashatóság miatt 10-el szorozva: kutya_pontok
kutya_helyszin['kutya_pontok'] = kutya_helyszin['rank_points'] / kutya_helyszin['kutya_team'] * 10
kutya_helyszin['kutya_pontok'] = kutya_helyszin['kutya_pontok'].round(2)

kutya_helyszin

,place,rank_points,kutya_team,kutya_pontok
0,BCCCZ_Prostejov,190,39,48.72
1,Heroltice,180,30,60.00
2,CrazyCup_NoveMesto,162,29,55.86
3,EagersCup_Zamberk,112,30,37.33
4,Tápiószentmárton,94,24,39.17
5,Flyvarors,80,16,50.00
6,Machale_Luonovice,80,16,50.00
7,Krypky_Litomsyl,76,20,38.00
8,Budeckej_Vodnany,68,17,40.00
9,Mistrovsy_Blsany,52,26,20.00


In [5]:
#Betöltjük a kutyák minden változóját tartalmazó táblát:
kutya_all = pd.read_csv('./../data/kutya_all_features.csv', index_col=[0])
kutya_all.head(1)

,r_min,r_median,r_mean,r_max,kutya_ev_honapok,Border/Other,min,median,mean,max,futasok_szama,valto_hiba,start_hiba,futas_hiba,összes_hiba,hiba_szazalek
Abby,0.0,0.22,0.31,1.25,55,Border,4.32,4.51,4.75,5.75,64.0,8.0,0.0,0.0,8.0,12.5


In [48]:
#Összekötjük a kutyák tábláját a kutyák helyezéseit és pontjait tartalmazó táblával
kutya_all_win = pd.merge(kutya_all, kutymok, left_index=True, right_index=True)

In [49]:
kutya_all_win.head(1)

,r_min,r_median,r_mean,r_max,kutya_ev_honapok,Border/Other,min,median,mean,max,futasok_szama,valto_hiba,start_hiba,futas_hiba,összes_hiba,hiba_szazalek,rank_points
Abby,0.0,0.22,0.31,1.25,55,Border,4.32,4.51,4.75,5.75,64.0,8.0,0.0,0.0,8.0,12.5,15


In [50]:
#Megnézzük hogyan korrigál minden mindennel is
kutya_all_win.corr()

/var/folders/_3/r7b6vqh57xg62bszfyf8bwhc0000gn/T/ipykernel_86779/2333562731.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  kutya_all_win.corr()


,r_min,r_median,r_mean,r_max,kutya_ev_honapok,min,median,mean,max,futasok_szama,valto_hiba,start_hiba,futas_hiba,összes_hiba,hiba_szazalek,rank_points
r_min,1.000000,0.561974,0.526513,0.039534,0.048852,0.079242,0.015125,0.017992,-0.090098,-0.338123,-0.300726,-0.138942,-0.148436,-0.312661,-0.047831,-0.306556
r_median,0.561974,1.000000,0.952744,0.575769,0.306839,0.581233,0.565958,0.607839,0.523680,-0.422748,-0.395066,-0.362835,0.153244,-0.328475,-0.013226,-0.342848
r_mean,0.526513,0.952744,1.000000,0.689677,0.268112,0.540822,0.548964,0.592764,0.552081,-0.452060,-0.320595,-0.314479,0.193568,-0.250206,0.117715,-0.374367
r_max,0.039534,0.575769,0.689677,1.000000,0.099084,0.319735,0.383999,0.428831,0.551522,-0.004329,0.139931,-0.142925,0.315291,0.203652,0.123155,0.101892
kutya_ev_honapok,0.048852,0.306839,0.268112,0.099084,1.000000,0.327181,0.419275,0.416367,0.210248,-0.275196,-0.446543,-0.299514,0.076888,-0.394031,-0.117069,-0.199834
min,0.079242,0.581233,0.540822,0.319735,0.327181,1.000000,0.961801,0.959138,0.547583,-0.507558,-0.570283,-0.269992,-0.073729,-0.543835,-0.162406,-0.385363
median,0.015125,0.565958,0.548964,0.383999,0.419275,0.961801,1.000000,0.994145,0.638210,-0.459790,-0.505517,-0.253456,0.038081,-0.457637,-0.127659,-0.346924
mean,0.017992,0.607839,0.592764,0.428831,0.416367,0.959138,0.994145,1.000000,0.688939,-0.464505,-0.486334,-0.244493,0.062089,-0.434257,-0.100697,-0.353756
max,-0.090098,0.523680,0.552081,0.551522,0.210248,0.547583,0.638210,0.688939,1.000000,-0.262222,-0.019974,0.037628,0.416568,0.079973,0.203406,-0.204211
futasok_szama,-0.338123,-0.422748,-0.452060,-0.004329,-0.275196,-0.507558,-0.459790,-0.464505,-0.262222,1.000000,0.685645,0.160269,-0.031653,0.625428,-0.280355,0.944803


### Mi az ami egy kutya szerzett pontjaival legjobban korrigál:
- hányszor versenyzett: +94% - minnél többet fut, annál több helyezést fog szerezni :)
- -38% a hiba százalékkal: minél pontosabb, annál több pontja van
- -38% a legjobb futási idővel: minél jobb a legjobb ideje (personal best) annál több helyezése van
- -37% a váltási idővel: stabilan, jól vált
- -35% a futási idővel: stabilan, jól fut
